In [35]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from io import StringIO

def __iter__(self): return 0

cos_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='BaOxz8_CpmxspQul_0ei3a8mn8gNQI6TGrNTiUdNrUX2',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud'
)

bucket = 'maternalhealthprogress-donotdelete-pr-yig6uibnvbij4a'
object_key = 'SDG-3-1-DATA-GOV.csv'

body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# ✅ Read the bytes, decode with correct encoding, and wrap in StringIO
raw_bytes = body.read()
decoded_string = raw_bytes.decode('cp1252')  # or try 'ISO-8859-1' if needed

# Convert to a file-like object
data_io = StringIO(decoded_string)

# Read into pandas
df = pd.read_csv(data_io)

# Preview
df.head()


,AreaID,AreaName,TimePeriod,Source,Sector,Subsector,Goal,Target,Indicator,Unit,SubgroupDimension,Subgroup,SubgroupOrder,DataValue,Footnote
0,IND,India,2011-12,NITI Aayog,No Poverty,Population,1: End poverty in all its forms everywhere,"1.1: By 2030, eradicate extreme poverty for al...",1.1.1: Poverty Gap Ratio,Percent,Urbanization,Rural,1,5.05,NaN
1,IND,India,2011-12,NITI Aayog,No Poverty,Population,1: End poverty in all its forms everywhere,"1.1: By 2030, eradicate extreme poverty for al...",1.1.1: Poverty Gap Ratio,Percent,Urbanization,Urban,2,2.7,NaN
2,IND,India,2011-12,NITI Aayog,No Poverty,Population,1: End poverty in all its forms everywhere,"1.2: By 2030, reduce at least by half the prop...",1.2.1: Proportion of population living below t...,Percent,Urbanization,Total,1,21.9,NaN
3,IND,India,2015-16,Ministry of Health and Family Welfare (Nationa...,No Poverty,Health Care,1: End poverty in all its forms everywhere,1.3: Implement nationally appropriate social p...,1.3.1: Percentage of households with any usual...,Percent,Location,Total,1,28.7,NaN
4,IND,India,2015-16,Ministry of Women and Child Development,No Poverty,Health Care,1: End poverty in all its forms everywhere,1.3: Implement nationally appropriate social p...,1.3.2: Number of beneficiaries under Integrate...,Number,Beneficiary type,Total Beneficiaries (Children 6 month-6 years ...,1,102131284,NaN


In [36]:
# Step 2: Drop unnecessary columns
df = df.drop(columns=['AreaID', 'AreaName', 'Footnote'])

# Step 3: Drop rows where target value is missing
df = df.dropna(subset=['DataValue'])

# Step 4: Convert target column to numeric (just in case)
df['DataValue'] = pd.to_numeric(df['DataValue'], errors='coerce')

# Step 5: One-hot encode all categorical features
df_encoded = pd.get_dummies(df.drop(columns=['DataValue']), drop_first=True)

# Step 6: Add target column back
df_encoded['DataValue'] = df['DataValue'].values

for col in df_encoded.columns:
    if df_encoded[col].dtype == 'uint8':
        df_encoded[col] = df_encoded[col].astype('int64')

# Step 7: Save the encoded file with headers and without index
df_encoded.to_csv('encoded_maternal_health2.csv', index=False)

print("✅ Encoded CSV created successfully.")


✅ Encoded CSV created successfully.


In [42]:
with open('encoded_maternal_health.csv', 'rb') as file_data:
    cos_client.upload_fileobj(
        Fileobj=file_data,
        Bucket='maternalhealthprogress-donotdelete-pr-yig6uibnvbij4a',  # your bucket name
        Key='encoded_maternal_health2.csv'  # new file name in cloud
    )

print("✅ File successfully uploaded to IBM Cloud Object Storage.")

✅ File successfully uploaded to IBM Cloud Object Storage.


In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='BaOxz8_CpmxspQul_0ei3a8mn8gNQI6TGrNTiUdNrUX2',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud')

bucket = 'maternalhealthprogress-donotdelete-pr-yig6uibnvbij4a'
object_key = 'encoded_maternal_health2.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,SubgroupOrder,TimePeriod_2011-12,TimePeriod_2011-20,TimePeriod_2013,TimePeriod_2013-15,TimePeriod_2014,TimePeriod_2014-16,TimePeriod_2015,TimePeriod_2015-16,TimePeriod_2015-16,...,Subgroup_Undertrials,Subgroup_Upper Primary,Subgroup_Urban,Subgroup_Urban Female,Subgroup_Urban Male,Subgroup_Violence against women,Subgroup_Voting Rights (Out of 11 international organizations),Subgroup_Wheat,Subgroup_Women(15-49 yrs),DataValue
0,1,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,5.050000e+00
1,2,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,2.700000e+00
2,1,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2.190000e+01
3,1,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,2.870000e+01
4,1,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,1.021313e+08
5,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,9.834239e+07
6,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,8.927693e+07
7,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,8.756067e+07
8,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,8.550515e+07
9,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,8.165472e+07
